
- **Course**: COMPSCI 589 Machine Learning, Fall 2024
- **Instructor:** Justin Domke
- **Assignment:** 6
- **Group work policy:** You are allowed to complete this homework in teams of at most 3 students. However, each student must submit their own individual .pdf and .zip files to Gradescope, and submit your own solutions to [Kaggle](https://www.kaggle.com/t/b22337fb72164946a0502214ad599c73)[.](https://www.kaggle.com/t/f4b02988497040ef9ec91152b3057669) List your team members at the beginning of your `report.pdf`. You may verbally discuss the assignment with course staff or students outside your group, subject to conditions in the [Syllabus](https://www.notion.so/Syllabus-89f719731ae84d01b2e0b7b66088e48b?pvs=21). Please also list any students or course staff (separately) at the beginning of your `report.pdf`. However, you may not *look, copy, or show* any part of another student's assignment. Copying any part of another assignment — even a single sentence or line of code — from anyone outside your team is considered plagiarism. We use sophisticated tools to detect this. Please do not do it.
- **AI usage policy**: Limited use is acceptable, subject to conditions in [Syllabus](https://www.notion.so/Syllabus-89f719731ae84d01b2e0b7b66088e48b?pvs=21).
- **Due date:** Friday Nov 8th, 6:00 PM
- **Submission instructions**:
    - For this assignment, you can prepare your solutions in any manner you like. You may use Markdown, LaTeX, handwrite them, or a combination of these. Regardless of how they are prepared, you must produce a single .pdf file for your report, and a zip file containing your source files and your code that you upload to Gradescope.
    - At the top of your .pdf file please include the following information:
        - Your name
        - Your email
        - (Please do *not* include your student ID.)
        - List anyone you discussed the assignment with, including course staff.
    - Additionally, you **must submit a .zip file** in Gradescope. Your .zip file should contain three things:
        - `report_src/` - A directory containing all source files for the report. This should include all files that you used to create your report .pdf (e.g. latex source files, plain text, images)
        - `code/` - A directory containing Python code for all parts of the assignment.
        - `code/run_me.py` - A single Python file that will generate all figures included in your report.
    - [Gradescope](https://www.gradescope.com/) instructions: submit your **PDF** to [Gradescope](https://www.gradescope.com/courses/835228), click on **Assignment 6**, then click on **Submit PDF** to upload a single pdf file, submit your **ZIP** to Gradescope, click on **Assignment 6 - Code** to upload a single zip file. After the pdf has been uploaded, mark the correct pages for each question.  Contact [Ke](mailto:kexiao@cs.umass.edu) if you do not have access to the [Gradescope](https://www.gradescope.com/) page.
    - For the purpose of late days, the **later** of your two submissions will be considered the submission time for your assignment. E.g., if you submit your .pdf on time, but the .zip is two days late, the assignment will be considered two days late.
    - Finally, the assignment asks you to upload predictions to [Kaggle](https://www.kaggle.com/t/b22337fb72164946a0502214ad599c73).  *Please create a Kaggle account using your [umass.edu](http://umass.edu) email address*. Otherwise, it is very difficult for us to map Kaggle submissions to students.

**Code:**

- For this assignment you **may** use the methods in [`sklearn.tree`](https://scikit-learn.org/stable/api/sklearn.tree.html), [`sklinear.linear_model`](https://scikit-learn.org/stable/api/sklearn.linear_model.html), [`sklearn.metrics`](https://scikit-learn.org/stable/api/sklearn.metrics.html) and [`sklearn.neighbors`](https://scikit-learn.org/stable/api/sklearn.neighbors.html).
- When using methods from [`sklinear.linear_model`](https://scikit-learn.org/stable/api/sklearn.linear_model.html), after training them you can call them via `decision_function()` only. Do **not** use `predict()` or `score()` or `predict_proba()`.
- You may also use [`sklearn.model_selection.KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)  — but **not** any other methods in [`sklearn.model_selection`](https://scikit-learn.org/1.5/api/sklearn.model_selection.html).
- If the assignment asks you to implement a particular function, you are expected to implement it yourself. If you find that the function is implemented somewhere within `sklearn` or `np` but not specifically banned above, your implementation should *not* consist of a call to that function. Example: If the question asked you to "implement the cross entropy loss", and you found that sklearn implemented "cross_entropy_loss" somewhere deep within the allowed packages, you should not just call the sklearn method, you should implement it yourself.

**Note**: If you have any clarifying questions, you may use Notion’s “comment” functionality to highlight text and ask your question. (Please let Ke know if for any reason you are not able to make comments.) If you have deeper questions, please come to office hours.


# Preliminaries

## Dataset

In this assignment you are given a set of 29x29 RBG images. There are four possible labels. Your goal will be to train a predictor to recognize what is in the image. Here are a few elements of the training data, shown as images:You are given a file `data.npz`. 

[data.npz](https://prod-files-secure.s3.us-west-2.amazonaws.com/500c04b7-1a6e-4d1d-9794-74feb55ce8f4/469e893e-55b2-4237-8eb3-508550579599/data.npz)

The data can be loaded as follows:

```python
stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]
# no Y_tst !
```

There are a total of 6000 training examples, and 1200 test examples, each with 2523 dimensions. Those dimensions correspond to 29x29x3 RBG images (29*29*3=2523). If you like, you can plot an example with the following code:

```python
from matplotlib import pyplot as plt
def show(x):
		img = x.reshape((3,31,31)).transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')
    plt.draw()
    plt.pause(0.01)
show(X_trn[7])
```

## Kaggle

Kaggle is a very popular platform for creating and running machine learning competitions. It allows the creators of competitions to evaluate submissions against a secret test set, ensuring that competitors cannot "cheat" by fine-tuning their model against the test set.

The makers of Kaggle also created a set of features for creating an "InClass" competition, perfect for classes such as 589! We created a competition for this assignment in which you are required to participate. However, don't worry! It's not truly a "competition" as much as it is a way to automatically evaluate your submissions and to familiarize you with the Kaggle platform. Again, to make it easier for us to grade, *please create a Kaggle account using your @umass.edu email address*.

The "competition" has two leaderboards: A public leaderboard and private leaderboard. The test set is split into two sets: A "public" set containing about 30% of the data and a "private" set containing the remaining 70%. In a normal competition, you can see how well your submission is performing against the public set. In theory, one could use brute force to find all of the correct answers. For this reason, in most competitions submissions are scored against the "private" set. This assignment will, at various points, ask you to report the performance of various solutions according to the public leaderboard.

To submit solutions to Kaggle, you will be required to submit a `.csv` file with two columns: an "Id" column and a "Category" column classifying the integer prediction for each element in `X_tst`. A sample solution using randomly predicted outputs can be generated as follows:

```python
import numpy as np
import csv

def write_csv(y_pred, filename):
    """Write a 1d numpy array to a Kaggle-compatible .csv file"""
    with open(filename, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Id', 'Category'])
        for idx, y in enumerate(y_pred):
            csv_writer.writerow([idx, y])

data = np.load('data.npz')
X_tst = data['X_tst']
y_pred = np.random.randint(0, 3, size=len(X_tst)) # random predictions
write_csv(y_pred, 'sample_predictions.csv')
```

You can use the write_csv helper function in your code if you find it helpful to ensure that your solution is in the correct format.

Note that the leaderboard shows *accuracy* whereas the assignment in some places asks for *classification error*. Note that these are related by

$$
\text{Classification Error} = 1 - \text{Accuracy},
$$

so it is easy to translate between the two.


## Part 1 - Classification

**Question 1** (5 points) Take a very small dataset with five scalar inputs:

$$
\begin{aligned}
x^{(1)} & = & 1.0 \\
x^{(2)} & = & 2.0 \\
x^{(3)} & = & 3.0 \\
x^{(4)} & = & 4.0 \\
x^{(5)} & = & 5.0 \\
\end{aligned}
$$

There are two possible labels, as shown below:

$$
\begin{aligned}
y^{(1)} & = & 1 \\
y^{(2)} & = & 1 \\
y^{(3)} & = & 0 \\
y^{(4)} & = & 0 \\
y^{(5)} & = & 1 \\
\end{aligned}
$$

For each of the following split points, what is the information gain? Show your work.

- Split at $x=0.5$
- Split at $x=1.5$
- Split at $x=2.5$
- Split at $x=3.5$
- Split at $x=4.5$
- Split at $x=5.5$

*Hint*: The information gain is defined to be the change in the entropy of the entropy that the split achieves. Let $I(p)=-\sum_v \hat{p}_v \log \hat{p}_v$ be the entropy of a distribution. We can calculate the distribution corresponding to any set of labels. For example, in the above dataset, $p=(0.4, 0.6).$ For a given split, the information gain is

$$
I(p)-\frac{N_1 I(p_1) + N_2 I(p_2)}{N},
$$

where $p$ is the distribution of labels before splitting, $N$ is the total number of points,  $p_1$ and $p_2$ are the distributions of labels in the two datasets that result from splitting, and $N_1$ and $N_2$ are the number of points in the two datasets that result from splitting.

*Hint 2:* Remember that when calculating entropies we interpret $0 \log 0$ as being equal to $0$. (We do this because $\lim_{x \rightarrow 0+} x \log x = 0$.)

# SOLN
Given that the information gain is given by $I(p)-\frac{N_1 I (p_1) + N_2 I(p_2)}{N}$, where $$I(p)=-\sum_{v}\hat{p_v}$$

We need to find the 'base entropy' without any splitting rules. This is $I(p)=-(0.4log_2(0.4)+0.6log_2(0.6))=0.9709$

First, we use the splitting rule $x=0.5$, which gives us our initial section. No information gain.

Next, we use $x=1.5$, which gives us two sections. The first corresponds to x1, and the second corresponds to x2,x3,x4,x5. $I(p_1)=-1 \log_2 1=0$

$I(p_2)=-(0.5log_2(0.5)+0.5log_2(0.5))=1$ The change in information then $0.9709-\frac{4}{5}=0.1709$

For $x=2.5$ we get the first two elements. $I(p_1)=-(1 log_2 1)=0$ and the second section is $I(p_2)=-(\frac{1}{3} log_2(1/3) + \frac{2}{3} log_2(2/3))=0.9182$

This gives us an information gain of $0.9709 - \frac{3\cdot 0.9182}{5}=0.41998$

For $x=3.5$ we get the first three elements. $I(p_1)=0.9182$ and $I(p_2)=1$

Our information gain is $0.9709 - \frac{0.9182 * 3 + 1 * 2}{5} = 0.01998$

For $x=4.5$, $I(p_1)=1$ and $I(p_2)=0$ 

Our information gain is $0.9709 - \frac{1*4}{5}=0.1709$ which is 

For $x=5.5$ our information gain is the same as for $x=0.5$ because no data is actually being split.

In [1]:
import numpy as np

stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]
X_train_flattened = X_trn.reshape(len(X_trn), -1)


**Question 2** (8 points) Write your own class of classification stump `ClassificationStump`, with the following signature:

```python
class ClassificationStump():
    def __init__(self):
        return

    def fit(self, X_trn, y_trn):
        # do stuff here

        self.model = (dim, thresh, c_left, c_right)
        return

    def predict(self, X_val, y_val):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)

        return y_pred
```

where `X_trn` is a 2D array of training inputs, and `y_trn` is a 1D array of training outputs. Your model `predict` function should predict the class to be `c_left` if `x[dim]<=thresh` and `c_right` otherwise, with `self.model` . Your model `fit` function should find values `dim`, `thresh`, `c_left`, and `c_right` to be used in the `predict` function such that the classification error is minimized:

$$
\text{Classification Error} = \frac{1}{N} \sum_{i=1}^{N} \mathbb{1}(y_i \neq \hat{y}_i)
$$

Where 

- N is the total number of training samples.
- $y_i$ is the true class label.
- $\hat{y}_i$ is the predicted class label.
- $\mathbb{1}(y_i \neq \hat{y}_i)$ is the indicator function, evaluates to 1 if $y_i \neq \hat{y}_i$, and 0 otherwise.

Train your `ClassificationStump` on the training data, and report the training classification error. 

In [2]:

class ClassificationStump():
    def __init__(self):
        return

    def fit(self, X_trn, y_trn):
        # do stuff here
        best_loss = np.inf

        N, d = X_trn.shape
        dim = d

        for feature in range(d): # Extract features
            sorted_idxs = np.argsort(X_trn[:, feature]) # The use of sorting helps find splits better
            X_sorted = X_trn[sorted_idxs]

            for idx in range(1, N): # loop over samples
                # The threshold will be determined between two succesive points, reducing the amount of compute needed
                curr_thresh = (X_sorted[idx - 1, feature] + X_sorted[idx, feature]) / 2
                left_mask = X_trn[:, feature] <= curr_thresh
                right_mask = X_trn[:, feature] > curr_thresh
                
                y_left = np.mean(y_trn[left_mask])
                y_right = np.mean(y_trn[right_mask])

                err_left = np.sum((y_trn[left_mask] - y_left) **2)
                err_right = np.sum((y_trn[right_mask] - y_right) **2 )
                err = err_left + err_right

                if err < best_loss:
                    dim = feature # Store the current dimension that we are splitting our data across
                    best_loss = err_left + err_right 
                    thresh = curr_thresh # New splitting rule
                    c_left = y_left
                    c_right = y_right


            self.model = (dim, thresh, c_left, c_right)
            return

    def predict(self, X_val, y_val):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)
        y_pred = []
        dim = self.model[0]
        thresh = self.model[1]
        c_left = self.model[2]
        c_right = self.model[3]
        for idx, x in enumerate(X):
            y_pred[idx] =  c_left if x[dim] <= thresh else c_right
        return y_pred


**Question 3** (8 points) Train 6 different classification trees on the image data, with each of the following maximum depths: {1,3,6,9,12,14}. (Do not apply any other restriction when growing the tree, you may use `sklearn.tree` for this question). Use the trees to predict on the training data, and report the classification error as a 6x1 table. You should have one row for each max_depth and the corresponding training error. 

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

depths = [1,3,6,9,12,14]
errors = [] 

# flatten 3D data to 2D
# if X_trn shape is (num_samples, height, width, channels) or (num_samples, height, width)

# train and calculate training error for each max_depth
for depth in depths:
    model = DecisionTreeClassifier(max_depth=depth)
    model.fit(X_train_flattened, y_trn)
    y_pred_train = model.predict(X_train_flattened)
    
    # calculate training error
    trn_error = 1 - accuracy_score(y_trn, y_pred_train)
    errors.append(trn_error)

# display errors as 6x1 table
for depth, error in zip(depths, errors):
    print(f"max_depth={depth}, training error={error:.4f}")

max_depth=1, training error=0.6433
max_depth=3, training error=0.5515
max_depth=6, training error=0.4033
max_depth=9, training error=0.2497
max_depth=12, training error=0.1293
max_depth=14, training error=0.0712



**Question 4** (5 points) Consider a classification tree with a maximum depth of $M$, trained on data width $D$ dimensions. What is the time complexity to evaluate that classification tree on a single new input? Give an answer (Something like "order of $\log(M) \sqrt{D}$") and explain in at most 3 sentences why your answer is correct.

# Ans
The time complexity is just O(M) since the number of dimensions does not relate to the number of computations that need to be done. That is purely defined by the depth of the tree.

**Question 5** (5 points) Take a dataset with $N$ elements each with $D$ dimensions. What is the time complexity to train a classification stump? Give an answer and explain why it's correct in at most 3 sentences.

# Ans
A classification stump is a form of decision tree with depth M=1. Thus to train the stump over a dataset, we need to compare each point with all other points for all dimensions once. This gives us a complexity of $D(N(N-1))$ which is order $O(DN^2)$ in big O


**Question 6** (10 points) Train a linear model with logistic loss, and ridge regularization, on the image data. That is, find w to minimize 

$$
\sum_{n=1}^N L(y^{(n)},W x^{(n)}) + \lambda \Vert W \Vert^2.
$$

where L is the logistic loss.

For this question, you need to train 5 different models with 5 different regularization constant $\lambda \in \{10^{-1},1,10,100, 1000\}$, and report the logistic loss and classification error on each one of the 5 models. Organize your numbers as a 5x2 table, with one row for each value of $\lambda$ and one column for the logistic loss and the other column for the classification error. Again, you are allowed to use `sklearn.linear_model`. but you are not permitted to use `predict()` or  `predict_proba()` , but `decision_function()` is OK. 

**Note**: To avoid the convergence warning from showing up during training, set the following arguments: `max_iter=10000`, `tol=0.001`. This will increase the running time for your code, but it should finish within ~5 minutes. You can use lower `max_iter` for debugging purposes. 

**Hint**: The parameter `C` used in skelarn does not translate directly to `$\lambda$` used in our objective function, choose `C` carefully so that it matches the  `$\lambda$` we use. 

(**Hint**: Here $W$ is a matrix, so the ridge regularizer is $\Vert W \Vert^2 = \sum_{v=1}^V \sum_{i=1}^D W_{vd}^2$. This is technically known as the Frobenius norm. (But you shouldn’t need to implement this since `sklearn` does it for you if you use ridge regularization with multiclass linear predictors.)

In [4]:
print(y_trn)

[1 3 1 ... 3 1 2]


In [5]:
from sklearn.linear_model import LogisticRegression

# lambdas given in question
lambdas = [0.1, 1, 10, 100, 1000]

# C = 1/lambda in sklearn
Cs = [1/lam for lam in lambdas]

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    exp_scores = np.exp(z)
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

def logistic_loss(scores, y_true):
    pred = softmax(scores)
    correct_class = pred[np.arange(len(y_true)), y_true]
    return -np.mean(np.log(correct_class))

results = []
for C in Cs:
    model = LogisticRegression(C=C, max_iter=1000, tol=0.001)
    model.fit(X_train_flattened, y_trn)
    
    # Use decision_function() as specified
    scores = model.decision_function(X_train_flattened)
    loss = logistic_loss(scores, y_trn) 
    # For classification error:
    pred_classes = np.argmax(scores, axis=1)
    error = np.mean(pred_classes != y_trn)
    
    print(f"C={C}, lambda={1/C}, loss={loss:.4f}, error={error:.4f}")
    

C=10.0, lambda=0.1, loss=0.4830, error=0.1733
C=1.0, lambda=1.0, loss=0.5683, error=0.2063
C=0.1, lambda=10.0, loss=0.6878, error=0.2627
C=0.01, lambda=100.0, loss=0.7859, error=0.3025
C=0.001, lambda=1000.0, loss=0.8779, error=0.3367



**Question 7** (8 points) Train a K nearest neighbor model for each of the following possible values of `K: {1,3,5,7,9,11}`. (Warning: This question might take a significant amount of computational time. You may consider using the `n_jobs` option.) Report the classification error as a 6x1 table, with one row for each value of K, and one column for the classification error. Note: you may use `sklearn.neighbors` for this question.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
K = [1,3,5,7,9,11]      

for k in K:
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train_flattened, y_trn)
    y_pred_train = neigh.predict(X_train_flattened)


    # calculate training error
    trn_error = 1 - accuracy_score(y_trn, y_pred_train)
    errors.append(trn_error)

# display errors as 6x1 table
for k, error in zip(K, errors):
    print(f"num neighbors={k}, training error={error:.4f}")

num neighbors=1, training error=0.6433
num neighbors=3, training error=0.5515
num neighbors=5, training error=0.4033
num neighbors=7, training error=0.2497
num neighbors=9, training error=0.1293
num neighbors=11, training error=0.0712



**Question 8** (5 points) Consider a dataset with $N$ elements, each with $D$ dimensions. What is the time complexity to evaluate a K-nearest neighbors classifier? Give an answer and explain why it's correct in at most 3 sentences.


There are three components. Firstly, we need to calculate the distances for all points which is $ND$, then we need to sort them which is complexity $NlogN$. Finally, we need to look at K points to decide what class our point is, this is $K$. In total, our complexity is $O(ND + NlogN + K)$


## Part 2 - Cross Validation


**Question 9** (8 points) In this question, you need to write your own K-Fold cross validation function `cross_validation()` with the following signature:

```python
from sklearn.model_selection import KFold

def cross_validation(classifier, X_trn, y_trn, n_folds=5):
    # do stuff here
    
    
    # Return the paired (model and error) for all folds
    return [(model1, error1), (model2, error2), ..., (modelK, errorK)]
    

# Usage for the cross_validation function:
classifier = classifier_class(**args)
outputs = cross_validation(classifier, 
                           X_trn, 
                           y_trn, 
                           n_folds=N)
avg_error = np.mean([out_[1] for out_ in outputs])
```

Where you have `classifier` object to be trained, and `X_trn`, `y_trn` as input data, `n_folds` as the number of folds to split the data (`X_trn`, `y_trn`) for cross validation. The `classifier` object is expected to have functions `fit()` and  `predict()`. And the only sklearn function you are allowed to use in the cross_validation function is `sklearn.model_selection.KFold`, other imports are not allowed. The `classifier` are expected to have the following signature for `fit()` and `predict()` functions: 

```python
class Classifier():
    def __init__(self, **args):
        ...
        return

    def fit(self, X_trn, y_trn):
        ...
        # self.model is stored
        return

    def predict(self, X_val, y_val):
        ...
        # self.model is used
        return y_pred
```

In the `cross_validation` function, you should split the data into K folds ($\text{fold}_1$, $\text{fold}_2$, …, $\text{fold}_K$), then train $K$ models where $\text{model}_i$ is trained with {$\text{fold}_1$, $\text{fold}_2$, …, $\text{fold}_{i-1}$, $\text{fold}_{i+1}$, …, $\text{fold}_{K}$} as the training data and {$\text{fold}_i$} as the validation data to report the `out-of-sample` classification error on. Then return the trained K models and the corresponding out-of-sample `classification error`.

## qs
Write my own cross validation function which can use KFold

Classifier is an object which can be trained and perform inference (e.g. it is a model)

Not clear what methods can be used for the classifier... how to abstract this to an arbitrary model?

In [18]:
from sklearn.model_selection import KFold

def cross_validation(classifier, X_trn, y_trn, n_folds=5):
    models = []
    errors = []
    kf = KFold(n_splits=n_folds)
    
    for train_index, test_index in kf.split(X_trn):
        # Train model on K-1 folds
        classifier.fit(X_trn[train_index], y_trn[train_index])
        
        # Test on held-out fold
        y_pred = classifier.predict(X_trn[test_index])

        error = sum(y_pred != y_trn[test_index])/len(y_pred)
        
        models.append(classifier)
        errors.append(error)
    
    return list(zip(models, errors))


**Quesiton 10** (8 points) Train 6 different classification trees with `sklearn` modules on the image data, with each of the following maximum depths: `{1,3,6,9,12,14}`, like in `Q3`, but for this question, you should train it with 5-Fold cross-validation developed in `Q9`, estimate the mean on the out-of-sample (generalization) classification error, and report it as a 6x6 table. You should have one row for each `max_depth`, one column for each fold, and the last column for the mean over all 5 folds. (Do not apply any other restriction when growing the tree.) 

In [12]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

depths = [1, 3, 6, 9, 12, 14]
all_errors = []  # will store errors for each depth

# Train and get cross-validation errors for each depth
for depth in depths:
    model = DecisionTreeClassifier(max_depth=depth)
    outputs = cross_validation(model, X_train_flattened, y_trn, n_folds=5)
    
    # Get errors for all folds at this depth
    fold_errors = [out[1] for out in outputs]
    # Add mean error to create the 6th column
    fold_errors.append(np.mean(fold_errors))
    
    all_errors.append(fold_errors)

# Print 6x6 table
print("max_depth  Fold1    Fold2    Fold3    Fold4    Fold5    Mean")
print("-" * 60)
for depth, errors in zip(depths, all_errors):
    print(f"{depth:8d}  {errors[0]:.4f}  {errors[1]:.4f}  {errors[2]:.4f}  {errors[3]:.4f}  {errors[4]:.4f}  {errors[5]:.4f}")

max_depth  Fold1    Fold2    Fold3    Fold4    Fold5    Mean
------------------------------------------------------------
       1  0.6542  0.6617  0.6358  0.6375  0.6300  0.6438
       3  0.5275  0.5692  0.5650  0.5475  0.5400  0.5498
       6  0.4842  0.4933  0.4942  0.4833  0.4817  0.4873
       9  0.4633  0.4925  0.4733  0.4667  0.4583  0.4708
      12  0.4567  0.4850  0.4750  0.4642  0.4733  0.4708
      14  0.4825  0.5025  0.4667  0.4667  0.4692  0.4775




**Question 11** (8 points) Train a linear model with logistic loss, and ridge regularization, on the image data, like in `Q6`. But for this question, you should train it with 5-fold cross-validation developed in `Q9`, estimate the mean on the out-of-sample (generalization) classification error, and report it as a 5x6 table. You should have one row for each of the 5 different regularization constant $\lambda \in \{10^{-1},1,10,100, 1000\}$, one column for each fold, and the last column for the mean over all 5 folds. You can use `sklearn` modules for this question.

In [29]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# lambdas given in question
lambdas = [0.1, 1, 10, 100, 1000]
# C = 1/lambda in sklearn
Cs = [1/lam for lam in lambdas]
all_errors = []

# For each regularization strength
for C in Cs:
    model = LogisticRegression(C=C, max_iter=1000, tol=0.001)
    outputs = cross_validation(model, X_train_flattened, y_trn, n_folds=5)
    
    # Get errors for all folds at this C value
    fold_errors = [out[1] for out in outputs]
    # Add mean error to create the 6th column
    fold_errors.append(np.mean(fold_errors))
    
    all_errors.append(fold_errors)

# Print 5x6 table
print("lambda    Fold1    Fold2    Fold3    Fold4    Fold5    Mean")
print("-" * 60)
for lam, errors in zip(lambdas, all_errors):
    print(f"{lam:8.1f}  {errors[0]:.4f}  {errors[1]:.4f}  {errors[2]:.4f}  {errors[3]:.4f}  {errors[4]:.4f}  {errors[5]:.4f}")

lambda    Fold1    Fold2    Fold3    Fold4    Fold5    Mean
------------------------------------------------------------
     0.1  0.4500  0.4717  0.4708  0.4508  0.4450  0.4577
     1.0  0.4167  0.4342  0.4358  0.4308  0.4158  0.4267
    10.0  0.3842  0.4017  0.3967  0.3975  0.3458  0.3852
   100.0  0.3392  0.3800  0.3450  0.3617  0.3283  0.3508
  1000.0  0.3458  0.3700  0.3525  0.3558  0.3325  0.3513



**Question 12** (8 points) ****Write ****your own K nearest neighbor classifier class, with the following signature: 

```python
class KNNClassifier():
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        return

    def fit(self, X_trn, y_trn):
        # Just store X_trn, y_trn
        self.model = (X_trn, y_trn)

    def predict(self, X_val, y_val):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)

        return y_pred

knn = KNNClassifier(n_neighbors)
models = cross_validation(knn, X_trn, y_trn, n_splits=5)
# report on the models for each n_neighbors
```

where `X_trn` is a 2D array of training inputs, and `y_trn` is a 1D array of training outputs. Your `KNNClassifier.fit()` function just stores the training data which will be used in `KNNClassifier.predict()` function by accessing `self.model`. 

Then train a K nearest neighbor model (with your `KNNClassifier`) for each of the following possible values of K: `{1,3,5,7,9,11}`, like in `Q7`. But for this question, you should train it with 5-fold cross-validation developed in `Q9`, estimate the mean on the out-of-sample (generalization) classification error, and report it as a 6x6 table. You should have one row for each of the 6 `n_neighbors (K)`, one column for each fold, and the last column for the mean over all 5 folds. 

Hint: You should check the `classification error` against your own answers in `Q7` before using it for cross-validation training. 

In [21]:
class KNNClassifier():
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        
    def fit(self, X_trn, y_trn):
        self.X_trn = X_trn
        self.y_trn = y_trn
        
    def predict(self, X_val):
        assert hasattr(self, "X_trn"), "No fitted model!"
        
        # Vectorized distance computation for all points at once
        # Expand (a-b)^2 = a^2 + b^2 - 2ab
        a2 = np.sum(X_val**2, axis=1, keepdims=True)
        b2 = np.sum(self.X_trn**2, axis=1)
        ab = X_val @ self.X_trn.T
        
        distances = np.sqrt(a2 + b2 - 2*ab)
        
        # Get k nearest neighbors for all points at once
        k_indices = np.argsort(distances, axis=1)[:, :self.n_neighbors]
        
        # Get predictions for all points
        k_nearest_labels = self.y_trn[k_indices]
        y_pred = np.array([np.bincount(labels).argmax() 
                          for labels in k_nearest_labels])
        
        return y_pred

In [22]:
# Run cross-validation
K = [1, 3, 5, 7, 9, 11]
all_errors = []

for k in K:
    model = KNNClassifier(n_neighbors=k)
    outputs = cross_validation(model, X_train_flattened, y_trn, n_folds=5)
    
    # Get errors for all folds at this k value
    fold_errors = [out[1] for out in outputs]
    # Add mean error to create the 6th column
    fold_errors.append(np.mean(fold_errors))
    
    all_errors.append(fold_errors)

# Print 6x6 table
print("K    Fold1    Fold2    Fold3    Fold4    Fold5    Mean")
print("-" * 60)
for k, errors in zip(K, all_errors):
    print(f"{k:8.1f}  {errors[0]:.4f}  {errors[1]:.4f}  {errors[2]:.4f}  {errors[3]:.4f}  {errors[4]:.4f}  {errors[5]:.4f}")

K    Fold1    Fold2    Fold3    Fold4    Fold5    Mean
------------------------------------------------------------
     1.0  0.4700  0.4583  0.4575  0.4408  0.4575  0.4568
     3.0  0.4825  0.4583  0.4658  0.4542  0.4867  0.4695
     5.0  0.4550  0.4417  0.4517  0.4325  0.4650  0.4492
     7.0  0.4383  0.4292  0.4467  0.4267  0.4483  0.4378
     9.0  0.4333  0.4258  0.4333  0.4333  0.4450  0.4342
    11.0  0.4308  0.4350  0.4408  0.4275  0.4308  0.4330



**Question 13** (8 points) Choose the model from `Q10`, `Q11`, and`Q12` that you think will perform best on the public leaderboard (So 3 models in total). Make predictions for the test data and upload your predictions to [Kaggle](https://www.kaggle.com/t/b22337fb72164946a0502214ad599c73). Report:

1. What classification model you chose
2. What hyper-parameters you chose (e.g. K, $\lambda$, etc.)
3. What was your estimated generalization error using 5-fold cross validation.
4. What accuracy you observed on the public part of the leaderboard.

1. Logistic Regression
2. Lambda=100
3. 35.08%
4. 0.6492

In [25]:

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    exp_scores = np.exp(z)
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

def get_classes(scores, y_true):
    pred = softmax(scores)
    correct_class = pred[np.arange(len(y_true)), y_true]
    return np.array(correct_class)

In [24]:

import numpy as np

stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]
X_test_flattened = X_tst.reshape(len(X_tst), -1)

In [26]:

from sklearn.linear_model import LogisticRegression

# lambdas given in question
lam = 100

# C = 1/lambda in sklearn
C = 1/lam


results = []
model = LogisticRegression(C=C, max_iter=1000, tol=0.001)
model.fit(X_train_flattened, y_trn)

# Use decision_function() as specified
scores = model.decision_function(X_train_flattened)
classes = get_classes(scores, y_trn) 

In [ ]:

import numpy as np
import csv

def write_csv(y_pred, filename):
    """Write a 1d numpy array to a Kaggle-compatible .csv file"""
    with open(filename, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Id', 'Category'])
        for idx, y in enumerate(y_pred):
            csv_writer.writerow([idx, y])

data = np.load('data.npz')
X_tst = data['X_tst']
y_pred = np.random.randint(0, 3, size=len(X_tst)) # random predictions
write_csv(y_pred, 'sample_predictions.csv')


**Question 14** (8 points). Like described in the assignment statement, the test set is split into **30% for the public leaderboard** and **70% for the private leaderboard**. You can see your model performance on the 30% public test set, while we will score your model performance on the 70% private leaderboard (which you have no access to) in this question. You can have **maximum 8 points** for your model performance on the **private leaderboard**. (No need to mark this question when submitting to Gradescope.)